### SHORT DESCRIPTION

* NCORE - numbers of core. For parallel pipeline NCORE > 1 (max NCORE = 8)
* DELETE_NOW - If True, bad reads (by filters) will be deleted after each filter
* IS_SHORT_FLANK - If True, ignore additional PRIMER and AD1 in R1
* CHAOS - If True, you're not sure that file(R1) is contained R1 and file(R2) is contained R2
* MIN_READ - Create own FIX and dbRIP file for each library (if NUM_READS >= MIN_READ)
* INSWINDOW_FIX - Delete reads if CURRENT_POS ~ [RESTRICT_SITE : IS + INSWINDOWS_FIX] (for INS_STRAND = +)
* SHIFT_RESTRICT_SITE - Mark reads if RESTRICT_SITE was found in [RESTRICT_SITE :  IS + SHIFT] (for INS_STRAND = +)
* SHIFT_MISS_PRIMER - Mark reads if PRIMER was found in [IS :  IS + SHIFT] (for INS_STRAND = +)
* SHIFT_MISS_RE -  Mark reads if PRIMER was found in [IS :  IS + SHIFT] (for INS_STRAND = +)
* RE_HAMMING - Delete reads RE_HAMMING_CURRENT >= RE_HAMMING
* FLANK_ERRORS - Delete reads FLANK_ERRORS_CURRENT >= FLANK_ERRORS
* REPEAT - Delete reads REPEAT_CURRENT >= REPEAT
* MISS_PRIMER - Delete reads MISS_PRIMER_CURRENT <= MISS_PRIMER
* MISS_RE - Delelte reads MISS_RE_CURRENT <= MISS_RE
* TEMPLATE_SWITCH_MD - Delete reads MDMATCH >= TEMPLATE_SWITCH_MD
* WINDOW - Window for megaclustering
* MAIN_FLAN_LEN - For template switch. TARGET_READS = PRIMER + RE + MAIN_FLAN_LEN(of R1)

In [1]:
import os
print(os.getcwd())
os.chdir('/mnt/c/Users/Komar/Lab/')
print(os.getcwd())

/mnt/c/Users/Komar/Lab/retroparty
/mnt/c/Users/Komar/Lab


In [2]:
#PATHWAYS
lll = '''
INPUTDIR = '/mnt/c/Users/Komar/Lab/InPutDir/artificial_data_multiUMI'
OUTPUTDIR = '/mnt/c/Users/Komar/Lab/OutPutDir/artificial_multiUMI'

MAPPER = 'bowtie2' #bwa or bowtie2
BOWTIE_INDEX = '/mnt/c/Users/Komar/Lab/UCSC_hg38/ref_bowtie2_index'
BWA_INDEX = '/mnt/c/Users/Komar/Lab/UCSC_hg38/ref_bwa_index'

ALU_REF_LIB = '/mnt/c/Users/Komar/Lab/retroparty/Alu_replibrary_hg38.txt'  # fixed and polymorphic Alu insertions 
REF_GENOME = '/mnt/c/Users/Komar/Lab/UCSC_hg38/genome.fa'
REPEATS_REF_LIB = '/mnt/c/Users/Komar/Lab/retroparty/repeats_hg38.tabular'  # all known fixed repeats

#SYS PARAMETRS
NCORE = 8  # threads

#PREPROCESSING
PRIMER = 'GAGCCACCGCGC'
SHIFT = 5
MIST = 1
AD1 = 'GCGTGCTGCGG'
AD2 = 'AGGGCGGT'
BLEN = 9
RE = 'CCGGCC'
RESTRICT_SITE = {'AGCT': 'CT', 'TCGA': 'GA'} # {'GTAC': 'TAC', 'CTAG': 'TAG'} # {'restrict site': 'r2_start'}
IS_SHORT_FLANK = False
CHAOS = True
TRIMM_N = 0
TRIMM_POLY_N = False
POLY_N_R1 = (15, 0.8, 7) # poly_n_win_size_r1, poly_n_th_r1, poly_n_shift_r1
POLY_N_R2 = (15, 0.8, 7) # poly_n_win_size_r2, poly_n_th_r2, poly_n_shift_r2
SKIP_SHORT_READS = 50
MID_MIST_SHORT_READS = (3, 2) # (r1 ,r2) max mismatches in AD2 or PRIMER which will remove from reads
END_MIST_SHORT_READS = (6, 6) # (r1 ,r2) min length of AD2 or PRIMER at the reads ends
PLACE_OF_SEARCH_TAIL = (None, None) # (r1 ,r2)
MIN_SEQ_LEN_AFTER_TRIMM = (25, 25) # (r1 ,r2)

#SAMFILTER
FLAGS = [99, 83, 147, 163] # + [355, 339, 403, 419]

R_SITE_DISTANCE = 1000

#FILTERS
FIX_WINDOW = 1
MIN_READ = 1
MAX_DIST = 1000
MIN_DIST = 0
INSWINDOW_FIX = 40
SHIFT_RESTRICT_SITE = 3 
SHIFT_MISS_PRIMER = 27
SHIFT_MISS_RE = 9

#THRESHOLDS
RE_HAMMING = 2
FLANK_ERRORS = 5
REPEAT = 0.8
MISS_PRIMER = 4
MISS_RE = 2
TEMPLATE_SWITCH_MD = 30

#METACLUSTERING
WINDOW = 100
MAIN_FLANK_LEN = 11 # template switch?  #'''

In [21]:
len('CATATGTAACTAACCTGCACAATGT')

25

In [39]:
import sys

#PATHWAYS
INPUTDIR = '/mnt/c/Users/Komar/Lab/InPutDir/test_L' # sys.argv[1]  # '/home/common/komarov.na/data/artificial_multi'
OUTPUTDIR = '/mnt/c/Users/Komar/Lab/OutPutDir/test_L_2'  # sys.argv[2]  #  '/home/common/komarov.na/results/artificial_multi'

MAPPER = 'bowtie2' #bwa or bowtie2
BOWTIE_INDEX = '/mnt/c/Users/Komar/Lab/UCSC_hg38/ref_bowtie2_index'
BWA_INDEX = '/mnt/c/Users/Komar/Lab/UCSC_hg38/ref_bwa_index'

ALU_REF_LIB = '/mnt/c/Users/Komar/Lab/UCSC_hg38/Human_L1_coord_hg38.txt'  # fixed and polymorphic target repeat insertions
REF_GENOME = '/mnt/c/Users/Komar/Lab/UCSC_hg38/genome.fa'
REPEATS_REF_LIB = '/mnt/c/Users/Komar/Lab/UCSC_hg38/repeats_hg38.tabular'  # all known fixed repeats

#SYS PARAMETRS
NCORE = 4  # threads

#PREPROCESSING
# LINE HaeIII
PRIMER = 'CATATGTAACTAACCTGCACAATGTGCACATGTACCCTAAAACTTAAAGTAT' #52 # 'GTGAGCCACCGCGC'
SHIFT = 5
MIST = 2
AD1 = 'AGGGCGTGGTGCGG' #'AGGGCGAG'  #'AGGGCGTGGTGCGG'
AD2 = 'AGGGCGG'  #'AGGGCGGT'
BLEN = 0
RE = 'AATAAA' #'CCGGCC'

#argumentList = sys.argv[3].split('_')
RESTRICT_SITE = {'GGCC': 'CC'}
            #{'GTAC':'AC', 'AGCT': 'CT'} # {'GTAC': 'TAC', 'CTAG': 'TAG'} # {'restrict site': 'r2_start'}
IS_SHORT_FLANK = True
CHAOS = True
TRIMM_N = 0
TRIMM_POLY_N = True
POLY_N_R1 = (15, 0.8, 30) # poly_n_win_size_r1, poly_n_th_r1, poly_n_shift_r1
POLY_N_R2 = (15, 0.8, 30) # poly_n_win_size_r2, poly_n_th_r2, poly_n_shift_r2
SKIP_SHORT_READS = 60
MID_MIST_SHORT_READS = (1, 2) # (r1 ,r2) max mismatches in AD2 or PRIMER which will remove from reads
END_MIST_SHORT_READS = (5, 5) # (r1 ,r2) min length of AD2 or PRIMER at the reads ends
PLACE_OF_SEARCH_TAIL = (None, None) # (r1 ,r2)
MIN_SEQ_LEN_AFTER_TRIMM = (25, 25) # (r1 ,r2)

#SAMFILTER
FLAGS = [99, 83, 147, 163] # + [355, 339, 403, 419]

R_SITE_DISTANCE = 1000

#FILTERS
FIX_WINDOW = 5 # 1
MIN_READ = 1
MAX_DIST = 1000
MIN_DIST = 0
INSWINDOW_FIX = 40 # 15 # 40
SHIFT_RESTRICT_SITE = 5 # 3
SHIFT_MISS_PRIMER = 60 # 27 # 20 # 12  # >= lenght of primer 
SHIFT_MISS_RE = 10 # 8
WHICH_TAIL = 3 # 5 if 5' repeat tail, 3 if 3' repeat tail

#THRESHOLDS
RE_HAMMING = 2
FLANK_ERRORS = 5
REPEAT = 0.8
MISS_PRIMER = 4
MISS_RE = 2
TEMPLATE_SWITCH_MD = 32 # do not used

#METACLUSTERING
WINDOW = 100
MAIN_FLANK_LEN = 14 # template switch

### <font color='blue'>1st round</font> 

#### 1. Raw FASTQ files preprocessing

In [40]:
import imp
import trimmRE
imp.reload(trimmRE)

trimmRE.main(inputdir = INPUTDIR,
              outputdir = os.path.abspath(OUTPUTDIR) + '/preprocessing/',
              primer = PRIMER,
              ad1 = AD1,
              ad2 = AD2,
              blen = BLEN,
              shift = SHIFT,
              mist = MIST,
              restrict_site = RESTRICT_SITE,
              re_part = RE,
              is_short_flank = IS_SHORT_FLANK,
              chaos = CHAOS,
              n_core = NCORE,
              trimm_n = TRIMM_N,
              trimm_poly_N = TRIMM_POLY_N,
              poly_n_r1 = POLY_N_R1,
              poly_n_r2 = POLY_N_R2,
              skip_short_reads = SKIP_SHORT_READS,
              mid_mist_short_reads = MID_MIST_SHORT_READS,
              end_mist_short_reads = END_MIST_SHORT_READS,
              place_of_search_tail = PLACE_OF_SEARCH_TAIL,
              min_seq_len_after_trimm = MIN_SEQ_LEN_AFTER_TRIMM)

start: BC60-L-1000_


#### 2. Map FASTQ by BWA

In [41]:
import bwamem
imp.reload(bwamem)

if MAPPER == 'bowtie2':
    mapper_execline = 'bowtie2 -p 4 -I 25 -X 1000 --dovetail'
    refway = BOWTIE_INDEX
else:
    if MAPPER != 'bwa':
        print('BWA')
    mapper_execline = 'bwa mem -t 4'
    refway = BWA_INDEX

bwamem.main(inputdir = os.path.abspath(OUTPUTDIR) + '/preprocessing/',
            outputdir = os.path.abspath(OUTPUTDIR) + '/mapping/',
            refway = refway,
            bwaline = mapper_execline)

bowtie2 -p 4 -I 25 -X 1000 --dovetail -x /mnt/c/Users/Komar/Lab/UCSC_hg38/ref_bowtie2_index -1 /mnt/c/Users/Komar/Lab/OutPutDir/test_L_2/preprocessing/BC60-L-1000__R1_good.fastq -2 /mnt/c/Users/Komar/Lab/OutPutDir/test_L_2/preprocessing/BC60-L-1000__R2_good.fastq -S /mnt/c/Users/Komar/Lab/OutPutDir/test_L_2/mapping/BC60-L-1000__.sam


#### 3. Filter SAM files

In [42]:
import samfilter_pysam
import gline
imp.reload(samfilter_pysam)
imp.reload(gline)

samfilter_pysam.main(inputdir = os.path.abspath(OUTPUTDIR) + '/mapping/',
                     outputdir = os.path.abspath(OUTPUTDIR) + '/table/',
                     flags = FLAGS, 
                     n_core = NCORE)

start: BC60-L-1000__


#### 4. Collapse reads by (chr, strand, pos)

In [43]:
import collapser
imp.reload(collapser)

collapser.main(inputdir = os.path.abspath(OUTPUTDIR) + '/table/',
               outputdir = os.path.abspath(OUTPUTDIR) + '/collapse_table/',
               target_re = RE,
               n_core = NCORE)

#### 5. Create own fix and polymorph ALUBASE

In [44]:
import exactmatch_fpALU
imp.reload(exactmatch_fpALU)

exactmatch_fpALU.main(inputdir = os.path.abspath(OUTPUTDIR) + '/collapse_table/',
                      outputdir = os.path.abspath(OUTPUTDIR) + '/ematch_table/',
                      replibrary = ALU_REF_LIB,
                      refway = REF_GENOME,
                      restrict_site = RESTRICT_SITE,
                      max_dist = MAX_DIST,
                      min_dist = MIN_DIST,
                      min_read = MIN_READ,
                      inswindow = FIX_WINDOW,
                      direction = WHICH_TAIL,
                      n_core = NCORE)

(801703, 8)
    CHR      START        END STRAND    NAME R_SITE  R_SITE_POS  IDX
0  chr1   67108753   67109046      +    L1P5   GGCC    67109047    0
1  chr1   25165803   25166380      +   L1MB5   GGCC    25167050    1
2  chr1   50331336   50332274      +    HAL1   GGCC    50332762    2
4  chr1  100662981  100669120      -   L1PA4   GGCC   100662661    4
5  chr1  176160333  176160805      +  L1PA8A   GGCC   176161047    5


#### 6. Delete fix and polymorph ALU from reads

In [45]:
import intersection_fpALU
imp.reload(intersection_fpALU)

intersection_fpALU.main(inputdir = os.path.abspath(OUTPUTDIR) + '/collapse_table/',
                        outputdir = os.path.abspath(OUTPUTDIR) + '/notfpALU_table/',
                        outputdir_fix = os.path.abspath(OUTPUTDIR) + '/fix_ins_table/',
                        replib_inputdir = os.path.abspath(OUTPUTDIR) + '/ematch_table/',
                        inswindow = INSWINDOW_FIX,
                        fix_ins = None,
                        n_core = NCORE)

###  <font color='blue'>Filters</font> 


#### 7. Find restrict site around intergration point

In [46]:
import misseq
imp.reload(misseq)

misseq.main(inputdir = os.path.abspath(OUTPUTDIR) + '/notfpALU_table/',
            outputdir = os.path.abspath(OUTPUTDIR) + '/filter_rsite/',
            refway = REF_GENOME,
            mseq = RESTRICT_SITE,
            mname = "R_SITE",
            shift = SHIFT_RESTRICT_SITE,
            n_core = NCORE)

#### 8. Find primer in flank

In [47]:
import misseq
imp.reload(misseq)

misseq.main(inputdir = os.path.abspath(OUTPUTDIR) + '/filter_rsite/',
            outputdir = os.path.abspath(OUTPUTDIR) + '/filter_primer/',
            refway = REF_GENOME,
            mseq = PRIMER,
            mname = 'MISS_P_HAMMING',
            shift = SHIFT_MISS_PRIMER,
            n_core = NCORE)

#### 9. Find part of RE in flank

In [48]:
import misseq
imp.reload(misseq)

misseq.main(inputdir = os.path.abspath(OUTPUTDIR) + '/filter_primer/',
            outputdir = os.path.abspath(OUTPUTDIR) + '/filter_re/',
            refway = REF_GENOME,
            mseq = None,
            mname = 'MISS_RE_HAMMING',
            shift = SHIFT_MISS_RE,
            n_core = NCORE)

#### 10. Intersect with repeats

In [49]:
import intersection_replib
imp.reload(intersection_replib)

intersection_replib.main(inputdir = os.path.abspath(OUTPUTDIR) + '/filter_re/',
                         outputdir = os.path.abspath(OUTPUTDIR) + '/filter_replib/',
                         repeatway = REPEATS_REF_LIB,
                         n_core = NCORE)

#### 10*. Apply filters

In [50]:
import trash_deleting
imp.reload(trash_deleting)

trash_deleting.main(inputdir = os.path.abspath(OUTPUTDIR) + '/filter_replib/',
                    outputdir = os.path.abspath(OUTPUTDIR) + '/pre_metatable/',
                    re_hamming = RE_HAMMING,
                    flank_errors = FLANK_ERRORS,
                    rsite = 'R_SITE',
                    repeat = REPEAT,
                    m_primer = MISS_PRIMER,
                    primer_name = 'P',
                    m_re = MISS_RE,
                    re_name = 'RE')

###  <font color='blue'>Metatable</font> 

#### 11. Metaclustering
At the clusterisation step all the coordinates of putative insertions from different datasets of the same individual are joined and sorted by value and chromosome. Next, coordinates are grouped with the window size of 100 nt. Putative somatic insertions are identified at this step: insertions with coordinates found in one dataset and absent from all other datasets from the same individual.

In [51]:
import metacluster
imp.reload(metacluster)

metacluster.main(inputdir = os.path.abspath(OUTPUTDIR) + '/pre_metatable/',
                 outputdir = os.path.abspath(OUTPUTDIR) + '/metatable_somatic/',
                 pcdir = os.path.abspath(OUTPUTDIR) + '/collapse_table/',
                 target_re = RE,
                 window = WINDOW,
                 blen = BLEN)

Paired:
BC60-L-1000__.txt <----- BC60-L-1000___pcread.txt


Unpaired:
NULL



In [52]:
import metacluster
imp.reload(metacluster)

metacluster.main(inputdir = os.path.abspath(OUTPUTDIR) + '/fix_ins_table/',
                 outputdir = os.path.abspath(OUTPUTDIR) + '/metatable_fix/',
                 pcdir = os.path.abspath(OUTPUTDIR) + '/collapse_table/',
                 target_re = RE,
                 window = WINDOW,
                 blen = BLEN)

Paired:
BC60-L-1000__.txt <----- BC60-L-1000___pcread.txt


Unpaired:
NULL



#### 12. Template switch

In [53]:
import template_switch
imp.reload(template_switch)

template_switch.main(inputdir = os.path.abspath(OUTPUTDIR) + '/metatable_somatic/',
                     outputdir = os.path.abspath(OUTPUTDIR) + '/result_somatic/',
                     primer = PRIMER,
                     main_flank_len = MAIN_FLANK_LEN,
                     template_switch_md = TEMPLATE_SWITCH_MD,
                     refway = BWA_INDEX,
                     bwaway = 'bwa')

bwa aln /mnt/c/Users/Komar/Lab/UCSC_hg38/ref_bwa_index /mnt/c/Users/Komar/Lab/OutPutDir/test_L_2/result_somatic/metatable.fastq > /mnt/c/Users/Komar/Lab/OutPutDir/test_L_2/result_somatic/metatable.sai
bwa samse /mnt/c/Users/Komar/Lab/UCSC_hg38/ref_bwa_index /mnt/c/Users/Komar/Lab/OutPutDir/test_L_2/result_somatic/metatable.sai /mnt/c/Users/Komar/Lab/OutPutDir/test_L_2/result_somatic/metatable.fastq > /mnt/c/Users/Komar/Lab/OutPutDir/test_L_2/result_somatic/metatable.sam


In [54]:
import template_switch
imp.reload(template_switch)

template_switch.main(inputdir = os.path.abspath(OUTPUTDIR) + '/metatable_fix/',
                     outputdir = os.path.abspath(OUTPUTDIR) + '/result_fix/',
                     primer = PRIMER,
                     main_flank_len = MAIN_FLANK_LEN,
                     template_switch_md = TEMPLATE_SWITCH_MD,
                     refway = BWA_INDEX,
                     bwaway = 'bwa')

bwa aln /mnt/c/Users/Komar/Lab/UCSC_hg38/ref_bwa_index /mnt/c/Users/Komar/Lab/OutPutDir/test_L_2/result_fix/metatable.fastq > /mnt/c/Users/Komar/Lab/OutPutDir/test_L_2/result_fix/metatable.sai
bwa samse /mnt/c/Users/Komar/Lab/UCSC_hg38/ref_bwa_index /mnt/c/Users/Komar/Lab/OutPutDir/test_L_2/result_fix/metatable.sai /mnt/c/Users/Komar/Lab/OutPutDir/test_L_2/result_fix/metatable.fastq > /mnt/c/Users/Komar/Lab/OutPutDir/test_L_2/result_fix/metatable.sam


# Result

### you can pull output table from 'result' folder in outputdir. Needed file is called 'metatable_ts.txt'

## Additional statistics 

### Init

In [30]:
import pandas as pd 
import os
from pathlib import Path

# 0 step
df = pd.DataFrame(columns=['art_1', 'art_2', 'art_3', 'art_4', 'art_5'])

In [31]:
df.loc['before'] = pd.Series({'art_1': 1000, 'art_2': 1000, 'art_3': 1000, 'art_4': 1000, 'art_5': 1000})

### 1 step: preprocessing

In [32]:
directory = os.path.abspath(OUTPUTDIR) + '/preprocessing/'
files = Path(directory).glob('*.txt')
i = 0
df.loc['trimm'] = pd.Series({'art_1': 0, 'art_2': 0, 'art_3': 0, 'art_4': 0, 'art_5': 0})
for file in files:
    with open(file, 'r') as f:
        df.loc['trimm'][i] = sum(1 for _ in f)
        i += 1
df

### 2 step: mapping

In [21]:
directory = os.path.abspath(OUTPUTDIR) + '/mapping/'
files = Path(directory).glob('*.sam')
i = 0
df.loc['mapping'] = pd.Series({'art_1': 0, 'art_2': 0, 'art_3': 0, 'art_4': 0, 'art_5': 0})
for file in files:
    with open(file, 'r') as f:
        df.loc['mapping'][i] = int(sum(1 if _[0] != '@' else 0 for _ in f) / 2)
        i += 1
df

### 3 step: filter sam files

In [22]:
directory = os.path.abspath(OUTPUTDIR) + '/table/'
files = Path(directory).glob('*.txt')
i = 0
df.loc['filter_sam_files'] = pd.Series({'art_1': 0, 'art_2': 0, 'art_3': 0, 'art_4': 0, 'art_5': 0})
for file in files:
    with open(file, 'r') as f:
        df.loc['filter_sam_files'][i] = sum(1 for _ in f) - 1
        i += 1
df

### 4 step: collapse reads by (chr, strand, pos)

In [23]:
directory = os.path.abspath(OUTPUTDIR) + '/collapse_table/'
files = Path(directory).glob('*humanread.txt')
i = 0
df.loc['grouping'] = pd.Series({'art_1': 0, 'art_2': 0, 'art_3': 0, 'art_4': 0, 'art_5': 0})
for file in files:
    with open(file, 'r') as f:
        df.loc['grouping'][i] = sum(1 for _ in f) - 1
        i += 1
df

### 6 step: delete fix and polymorph ALU from reads

In [24]:
directory = os.path.abspath(OUTPUTDIR) + '/notfpALU_table/'
files = Path(directory).glob('*.txt')
i = 0
df.loc['no_fpALU'] = pd.Series({'art_1': 0, 'art_2': 0, 'art_3': 0, 'art_4': 0, 'art_5': 0})
for file in files:
    with open(file, 'r') as f:
        df.loc['no_fpALU'][i] = sum(1 for _ in f) - 1
        i += 1
df

### 7 step: find restrict site around intergration point

In [25]:
directory = os.path.abspath(OUTPUTDIR) + '/filter_rsite/'
files = Path(directory).glob('*.txt')
i = 0
df.loc['no_rsite'] = pd.Series({'art_1': 0, 'art_2': 0, 'art_3': 0, 'art_4': 0, 'art_5': 0})
for file in files:
    with open(file, 'r') as f:
        df.loc['no_rsite'][i] = sum(1 for _ in f) - 1
        #print(file, df.loc['step7'][i])
        i += 1
df

### 8 step: find primer in flank

In [26]:
directory = os.path.abspath(OUTPUTDIR) + '/filter_primer/'
files = Path(directory).glob('*.txt')
i = 0
df.loc['no_flank_primer'] = pd.Series({'art_1': 0, 'art_2': 0, 'art_3': 0, 'art_4': 0, 'art_5': 0})
for file in files:
    with open(file, 'r') as f:
        df.loc['no_flank_primer'][i] = sum(1 for _ in f) - 1
        #print(file, df.loc['step8'][i])
        i += 1
df

### 9 step: find part of RE in flank

In [27]:
directory = os.path.abspath(OUTPUTDIR) + '/filter_re/'
files = Path(directory).glob('*.txt')
i = 0
df.loc['no_flank_re'] = pd.Series({'art_1': 0, 'art_2': 0, 'art_3': 0, 'art_4': 0, 'art_5': 0})
for file in files:
    with open(file, 'r') as f:
        df.loc['no_flank_re'][i] = sum(1 for _ in f) - 1
        i += 1
df

### 10 step: intersect with repeats

In [28]:
directory = os.path.abspath(OUTPUTDIR) + '/pre_metatable/'
files = Path(directory).glob('*.txt')
i = 0
df.loc['no_f_repeats'] = pd.Series({'art_1': 0, 'art_2': 0, 'art_3': 0, 'art_4': 0, 'art_5': 0})
for file in files:
    with open(file, 'r') as f:
        df.loc['no_f_repeats'][i] = sum(1 for _ in f) - 1
        i += 1
df

### 11 step: metaclustering

In [29]:
directory = os.path.abspath(OUTPUTDIR) + '/metatable_somatic/'
files = Path(directory).glob('*humanread.txt')

df.loc['metaclustering'] = pd.Series({'art_1': 0, 'art_2': 0, 'art_3': 0, 'art_4': 0, 'art_5': 0})
for file in files:
    with open(file, 'r') as f:
        line = f.readline()
        while line != '':
            line = f.readline()
            if line == '':
                break
            line = line.split('\t')[1]
            df.loc['metaclustering'][int(line[17]) - 1] += 1
df

### 12 step: template switch control

In [ ]:
directory = os.path.abspath(OUTPUTDIR) + '/result_somatic/'
files = Path(directory).glob('*_ts.txt')

df.loc['template_switch'] = pd.Series({'art_1': 0, 'art_2': 0, 'art_3': 0, 'art_4': 0, 'art_5': 0})
for file in files:
    with open(file, 'r') as f:
        line = f.readline()
        while line != '':
            line = f.readline()
            if line == '':
                break
            line = line.split('\t')[1]
            df.loc['template_switch'][int(line[17]) - 1] += 1
df

In [ ]:
df = df.transpose()

In [ ]:
df_mean = pd.DataFrame(df.mean(axis=0), columns=['num reads'])
df_mean['step'] = df_mean.index
df_mean

In [ ]:
df_percent = pd.DataFrame(df_mean)
df_percent['num reads'] = df_mean['num reads'] / df_mean['num reads'][5] * 100
df_percent

In [ ]:
import seaborn as sns


ax = sns.barplot(x=df_percent['step'].iloc[[5, 6, 7, 8, 9, 11]], 
                 y=df_percent['num reads'].iloc[[5, 6, 7, 8, 9, 11]], color='grey')

ax.bar_label(ax.containers[0])
ax.tick_params(axis='x', rotation=45)

In [ ]:
new_df = pd.DataFrame(df['before'], columns=['before', 'after_processing'])
new_df['after_processing'] = df['template_switch']
new_df

In [ ]:
res = pd.read_table(os.path.abspath(OUTPUTDIR) + '/result_somatic/'+'metatable_ts.txt')
res.head()